# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию расчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хранить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [173]:
# все импорты тут 

from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
from collections import Counter
import numpy as np

from scipy.sparse import csr_matrix, lil_matrix

import nltk
from nltk.tokenize import sent_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [174]:
# считываем корпус новостей Lenta
news = open('lenta.txt').read() 

In [175]:
# удаляем пунктуацию, токенизируем и приводим к нижнему регистру
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [176]:
# применим нормализацию к корпусу
norm_news = normalize(news)

In [177]:
# создадим частотный словарь 
vocab_news = Counter(norm_news)

In [178]:
# переведем абсолютные частоты в вероятности, разделив кол-во употреблений слова на общее число слов в корпусе
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [179]:
# ищем кол-во вхождений каждой n-граммы
def ngrammer(tokens, n):
    ngrams = []
    # проходимся по токенам
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [180]:
# токенезируем на предложения и добавляем теги вначале и в конце
sentences_news = [['<start>','<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(news)]

# берем отрывок из 10 предложений для расчета перплексии
test_sentences = sentences_news[-10:]
test_sentences = [' '.join(sent).replace('<start>', '').replace('<end>', '') for sent in test_sentences]

# убираем эти предложения из основной выборки
sentences_news = sentences_news[:-10]

Создадим частотные словари униграм, биграм и триграм

In [181]:
unigrams_news = Counter()
bigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    bigrams_news.update(ngrammer(sentence,2))
    trigrams_news.update(ngrammer(sentence, 3))

Частотный словарь для униграм:

In [182]:
unigrams_news.most_common(10)

[('<start>', 152668),
 ('<end>', 76334),
 ('в', 72401),
 ('и', 33289),
 ('на', 28432),
 ('по', 19489),
 ('что', 17031),
 ('с', 15918),
 ('не', 12701),
 ('из', 7727)]

Частотный словарь для биграм:

In [ ]:
bigrams_news.most_common(10)

[('<start> <start>', 76344),
 ('<start> в', 7972),
 ('<start> по', 6211),
 ('<start> как', 3738),
 ('риа новости', 3504),
 ('по словам', 1971),
 ('об этом', 1795),
 ('<start> однако', 1694),
 ('<start> на', 1643),
 ('что в', 1624)]

Частотный словарь для триграм:

In [ ]:
trigrams_news.most_common(10)

[('<start> <start> в', 7972),
 ('<start> <start> по', 6211),
 ('<start> <start> как', 3738),
 ('<start> <start> однако', 1694),
 ('<start> <start> на', 1643),
 ('<start> <start> об', 1619),
 ('<start> об этом', 1579),
 ('<start> <start> он', 1553),
 ('<start> по словам', 1549),
 ('сообщает риа новости', 1324)]

# Генерация текста с помощью трехграммной языковой модели

Матрица вероятностей:

In [183]:
matrix_news = lil_matrix((len(bigrams_news), 
                   len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}

id2bigram_news = list(bigrams_news)
bigram2id_news = {word:i for i, word in enumerate(id2bigram_news)}

for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2

    matrix_news[bigram2id_news[bigram], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     bigrams_news[bigram])

Функция генерации текста на основе трехграммной языковой модели:

In [184]:
def generate(matrix, id2word, id2bigram, bigram2id, n=250, start='<start> <start>'):
    text = []
    current_idx = bigram2id[start]
    
    for i in range(n):
        chosen = np.random.choice(list(range(matrix.shape[1])), 
                                  p=matrix[current_idx].toarray()[0])

        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
          current_idx = bigram2id[start]
        else:
          chunk = id2bigram[current_idx] + ' ' + id2word[chosen]
          chunk = ' '.join(chunk.split()[1:])
          current_idx = bigram2id[chunk]
    
    return ' '.join(text)

Сгенерируем несколько текстов с помощью модели

In [194]:
print(generate(matrix_news, id2word_news, id2bigram_news, bigram2id_news).replace('<end>', '\n'))

в проекте москва-сити-2000 
 в результате удара грузовик перевернулся раздавив еще два дня будет опрошено 80 тысяч человек 
 напомним что 10 апреля в москве 
 помимо взаимодействия с администрацией яковлева по блоку отечество вся россия правительство россии выразило заинтересованность в заводах daewoo проявила также и его состояние как тяжелое 
 договор по про 
 многих из них не намерены отказываться от собственной программы реформ по ряду одномандатных округов ни у россиян являются собственностью великобритании сегодня в час больше у чем у всех прочих проектов из средств приморского военного совхоза речинский надеясь что никто не пострадал 
 из 74 тысяч первично зарегистрированных в территориальных водах 10 танкеров подозреваемых в участии в бандформированиях в гудермесе по подозрению в причастности к совершенному накануне нападению на колонну подмосковного омона понесли конкретные лица которые занимались их переправкой за пределы взлетно-посадочной полосы получили незначительные повр

In [121]:
print(generate(matrix_news, id2word_news, id2bigram_news, bigram2id_news).replace('<end>', '\n'))

по его словам была проверена информация ряда сми о том что правительство россии внесло изменения в закон о выборах в грузинский парламент приняло участие в судьбе ветерана второй мировой войны и мира iwpr 
 в дни трагедии число доноров перевалило за миллиард долларов 
 департамент рекомендует гражданам российской федерации василий стародубцев в понедельник о совместном приобретении терпящей бедствие daewoo motor сообщает немецкая handelsblatt 
 из мировой практики 
 причины экономический кризис падение количества авиаперевозок увеличение цены авиатоплива и строгость новых европейских требований по вэбовкам 7 от юридических лиц с общей суммой требований в городских условиях сообщила телекомпания нтв сообщила что ей очень нравится походка владимира путина за содействие поездке делегации се в чечню вылетают эксперты совета европы передает интерфакс 
 момент когда машина ехала по улице гурьянова 19 эквивалентен взрыву 300-400 кг в либо большого объема работы прокуратура москвы расследующая

In [124]:
print(generate(matrix_news, id2word_news, id2bigram_news, bigram2id_news).replace('<end>', '\n'))

всего в здании городской администрации владимир пирожков были членами предвыборного штаба союза правых сил представитель фракции кпрф воздержались от голосования 
 прибывшие на место пожарные 
 осужденный моу цыжон стал одним из основных тем обсуждения на заседании отмечалось что вдействующем законодательстве россии не подтверждают что у чеченской стороны 
 в течение часа но потом когда люди возвращались с экскурсии занесло на скользкой дороге бортовой компьютер сбавляет мощность идущую на оформление букетов и икебан 
 министр также проинформировал о намеченном на 24 часа в основном сводится к непрерывному конвертированию текстовой новостной информации в течение часа летать кругами чтобы выработать излишки топлива 
 участники встречи заняли бескомпромиссную позицию в новом доме расположенном около местной больницы он начал раздавать автографы кто-то стукнул его по почте 
 принц уэльский получает доходы от владения герцогством корнуолл в размере 50 миллионов долларов в течение двух часо

## Перплексия

Функция для расчета перплексии:

In [186]:
def perplexity(probas):
    p = np.exp(np.sum(probas))
    N = len(probas)
    
    return p**(-1/N) 


Посчитаем усредненную перплексию для 10 предложений 

In [187]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})

In [188]:
probs = []

for sent in test_sentences:
  prob = []
  for word in normalize(sent):
    if word in probas_news:
        prob.append(np.log(probas_news[word]))
    else:
        prob.append(np.log(1/len(norm_news)))
  probs.append(prob)

In [197]:
prplxs = [] # тут хранятся значения перплексий для каждого из 10 предложений
for prob in probs:
  p = perplexity(prob)
  prplxs.append(p)

# усредненное значение перплексии для отрывка текста из 10 предложений
avg_perplexity = np.mean(prplxs)

In [198]:
avg_perplexity

11794.537887591401

Сравнение с языковой моделью на основе 3-грамм

In [191]:
probs_model = []

for sent in test_sentences:
  prob = []
  for ngram in ngrammer(['<start>', '<start>'] + normalize(sent) + ['<end>'], 3):
    word1, word2, word3 = ngram.split()
    bigram = word1 + ' ' + word2

    if bigram in bigrams_news and ngram in trigrams_news:
        prob.append(np.log(trigrams_news[ngram]/bigrams_news[bigram]))
    else:
        prob.append(np.log(0.00001))
    probs_model.append(prob)

In [195]:
prplxs_model = [] # тут хранятся значения перплексий для каждого из 10 предложений
for prob in probs_model:
  p = perplexity(prob)
  prplxs_model.append(p)

# усредненное значение перплексии для отрывка текста из 10 предложений
avg_perplexity_model = np.mean(prplxs_model)

In [196]:
avg_perplexity_model

13319.775135275695

Возможно, в рассчете перплексии есть ошибка, так как значение перплексии для языковой модели получилось выше, чем для корпуса.

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

**1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?**

В статье описаны два способа обработки несловарных слов:

1) Выбрать фиксированный словарь; на этапе нормализации преобразовать все несловарные слова (которых нет в фиксированном словаре) в токен неизвестного слова с помощью метки UNK (от англ. unknown word token). Рассчитать вероятности для UNK как для других слов в обучающей выборке. 
<br>
2) Неявно создать словарь на основе всей обучающей выборки, заменив малочастнотные слова на UNK 

**2. Что такое сглаживание (smoothing)?**

Сглаживание используется для избежания занулений вероятностей языковой моделью для слов, которые есть в словаре, но появились в неизвестном контексте. Для обработки таких слов мы отделяем небольшую часть значения вероятности у более частотных слов и присваиваем эту часть вместо нулевой вероятности. 